### Evaluation

In [73]:
import pandas as pd
import numpy as np 
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
from datetime import timedelta

In [74]:
#path to the mindsmall training data
training_dataset = '../MINDsmall_train'

validation_dataset = '../MINDsmall_train'

# not sure how to read these ones with header calues yet, as they are .vec files, and not .csv files
entity_embedding = pd.read_csv("../MINDsmall_train/entity_embedding.vec", sep='\t')
relation_embedding = pd.read_csv("../MINDsmall_train/relation_embedding.vec", sep='\t')


news = pd.read_csv("../MINDsmall_train/news.tsv",
    sep='\t',
    names=["newsId", "category", "subcategory", "title","abstract", "url", "title_entities","abstract_entities"]
)

# behaviors_data = pd.read_csv(
#     "MINDsmall_train/behaviors.tsv",
#     sep='\t',
#     names=["impressionId","userId","timestamp","click_history","impressions"]
# )
behaviors = pd.read_csv(
    "../MINDsmall_train/behaviors.tsv",
    sep='\t',
    names=["impressionId", "userId", "timestamp", "click_history", "impressions"],
    parse_dates=['timestamp'] # This line parses the 'timestamp' column into datetime objects
)

# Data preprocessing
behaviors = behaviors.sort_values(by='timestamp')
behaviors = behaviors.dropna(subset=['click_history'])

behaviors.head()
behaviors[behaviors['click_history'].str.contains("N55189")]

,impressionId,userId,timestamp,click_history,impressions
104697,104698,U8568,2019-11-09 00:10:50,N17933 N30344 N38491 N38198 N22217 N39074 N182...,N37108-0 N57099-0 N15134-0 N27845-0 N41881-0 N...
25110,25111,U68628,2019-11-09 00:17:06,N44579 N29177 N35009 N29718 N35120 N63650 N349...,N57099-0 N3491-0 N12523-1 N55645-0 N18378-0 N5...
133796,133797,U8568,2019-11-09 00:26:06,N17933 N30344 N38491 N38198 N22217 N39074 N182...,N52433-1 N15134-0 N57099-0 N64536-0 N50135-0 N...
121142,121143,U2906,2019-11-09 00:31:49,N51591 N19016 N25740 N459 N4834 N21242 N51892 ...,N3491-0 N58890-0 N62801-0 N47520-0 N62269-0 N5...
136962,136963,U85230,2019-11-09 00:39:16,N25740 N62931 N19594 N49944 N60615 N5153 N5625...,N62070-0 N39949-0 N37108-0 N45543-1 N3491-0 N6...
...,...,...,...,...,...
75007,75008,U50287,2019-11-14 23:38:53,N28818 N55189 N2203 N6233 N56207 N55031 N14439...,N29490-1 N41934-0
13609,13610,U91142,2019-11-14 23:39:37,N32979 N23571 N30708 N21457 N29718 N52897 N274...,N42767-0 N22975-0 N5940-1 N2960-0 N41934-0 N50...
87839,87840,U12239,2019-11-14 23:45:48,N6144 N64218 N49997 N22479 N14492 N5962 N6956 ...,N9047-0 N30290-0 N16854-0 N11930-0 N58656-0 N6...
75330,75331,U4536,2019-11-14 23:49:41,N37720 N55189 N59704 N13138 N55885 N16380 N184...,N27289-0 N23513-0 N48487-0 N17031-0 N15678-0 N...


In [75]:
# Take a look at the impression to user U13740
behaviors[behaviors['userId'] == "U13740"]

,impressionId,userId,timestamp,click_history,impressions
35262,35263,U13740,2019-11-09 05:59:43,N55189 N42782 N34694 N45794 N18445 N63302 N104...,N20020-0 N3737-0 N43202-0 N18708-0 N30125-0 N3...
0,1,U13740,2019-11-11 09:05:58,N55189 N42782 N34694 N45794 N18445 N63302 N104...,N55689-1 N35729-0
154836,154837,U13740,2019-11-13 15:27:40,N55189 N42782 N34694 N45794 N18445 N63302 N104...,N13907-0 N8509-0 N47061-0 N51048-0 N22417-0 N3...


### Hit Rate

In [76]:
# Filter clicks from the last two days

min_date = behaviors['timestamp'].min() + pd.Timedelta(hours=6)
max_date = min_date + pd.Timedelta(hours=3) 
print(f"Test start date: {min_date}")
print(f"Test end date: {max_date}")


filtered_behaviors = behaviors[(behaviors['timestamp'] >= min_date) & (behaviors['timestamp'] <= max_date)]

# Split data for training and evaluation: last day for test, previous day for training
train_end_date = max_date - timedelta(days=1)
train_data = filtered_behaviors[filtered_behaviors['timestamp'] <= train_end_date]
test_data = filtered_behaviors[filtered_behaviors['timestamp'] > train_end_date]

# Function to calculate hit rate
def calculate_hit_rate(news_id, actual, recommendations):
    total_hits = 0
    total_rows = 0
    
    matching_rows = actual[actual['click_history'].str.contains(news_id)]
    
    if not matching_rows.empty:
        for index, row in matching_rows.iterrows():
            if any(newsId in row['click_history'] for newsId in recommendations):
                total_hits += 1
            total_rows += 1
    
    hit_rate = total_hits / total_rows if total_rows > 0 else 0
    print(f"Total_hits: {total_hits} / {total_rows}")
    print(f"Hit rate: {hit_rate*100:.2f}%")

# Dummy recommendation of 5 news articles
recommendations = ['N21746', 'N44110', 'N63248', 'N21623', 'N10343']       

calculate_hit_rate('N55189', test_data, recommendations)


Test start date: 2019-11-09 06:00:19
Test end date: 2019-11-09 09:00:19
Total_hits: 14 / 201
Hit rate: 6.97%


### Precision@K

Precision at K measures the proportion of relevant items among the top K items. Out of 5 recommended items, the user interacted with 8.69% of items. 

In [103]:
min_date = behaviors['timestamp'].min() + pd.Timedelta(hours=6)
max_date = min_date + pd.Timedelta(hours=3) 
print(f"Test start date: {min_date}")
print(f"Test end date: {max_date}")

filtered_behaviors = behaviors[(behaviors['timestamp'] >= min_date) & (behaviors['timestamp'] <= max_date)]

# Split data for training and evaluation: first 6 hours for training, next 3 hours for training
train_end_date = max_date - timedelta(days=1)
train_data = filtered_behaviors[filtered_behaviors['timestamp'] <= train_end_date]
test_data = filtered_behaviors[filtered_behaviors['timestamp'] > train_end_date]


def precision_at_k(recommendations, click_history, k=5):
    # Convert the click history string to a list of clicked articles
    clicked_articles = click_history.split()
    # Take only the first k recommendations (if there are more than k)
    recommendations = recommendations[:k]
    # Count the number of recommended articles that are actually clicked
    num_clicked = sum(article in clicked_articles for article in recommendations)
    # Calculate precision at k
    precision = num_clicked / k if k > 0 else 0
    return precision


def calculate_precision_at_k(recommendations, actual, k=5):
    precisions = []
    
    # Iterate over each row of the recommendations DataFrame
    for recommendation in recommendations:
        for index, row in test_data.iterrows():
            count = 0
            click_history = row['click_history']
            click_history_split = click_history.split()     #click_history_split = test_data['click_history'].str.split()
            for news_id in click_history_split:
                if news_id in recommendation: 
                    count += 1
            precisions.append(count)
        
        # Calculate precision at k for the recommendations and click history
        precision = precision_at_k(recommendations, click_history, k)
        precisions.append(precision)
    
    # Calculate the average precision across all precisions
    average_precision = sum(precisions) / len(precisions) if len(precisions) > 0 else 0
    
    print(f"Average precision: {sum(precisions)} / {len(precisions)}")
    print(f"Precision at k={k}: {average_precision*100:.2f}%")

recommendation_lists = [
    ['N27448', 'N57737', 'N14881', 'N38042', 'N55576'],
    ['N27448', 'N48736', 'N46169', 'N19184', 'N15023']
]

# Example usage:
calculate_precision_at_k(recommendation_lists, test_data, k=5)


Test start date: 2019-11-09 06:00:19
Test end date: 2019-11-09 09:00:19
Average precision: 525.0 / 6040
Precision at k=5: 8.69%
